In [2]:
import os, re, pickle
from pprint import pprint
import numpy as np
import pandas as pd
corpora = ['Kamchatka', 'Sebjan']

Здесь собираю всё то же, что раньше, но отмечая, кто это говорит

Сначала видимо надо сделать сырой список морфем

In [11]:
def define_speaker(sent):
    '''возвращает спикера предложения'''
    if 'ELANParticipant' in sent:
        speaker = sent['ELANParticipant'][0]
    else:
        try:
            speaker = re.search('Speaker\sabbreviation = (...)', content[doc]['meta']).group(1)
        except:
            speaker = '?'
#             errors.append(doc)
    speaker = speaker.strip(',.?')
    if len(speaker) == 0 or ':' in speaker:
        speaker = '?'
    return speaker

def extract_morphs(corpus):
    with open('{}.pickle'.format(corpus), 'rb') as f:
        content = pickle.load(f)
    morphs = []
    for doc in content:
        for sent in content[doc]['text']:
            if 'mb' in sent and 'ge' in sent and 'ps' in sent:
                speaker = define_speaker(sent)
                for i in range(len(sent['mb'])):
                    morphs.append(tuple(['ROOT', 'ROOT', 'ROOT', speaker]))
                    ps = sent['ps'][i][0]
                    for l in range(1, len(sent['mb'][i])):
                        try:
                            morphs.append((sent['mb'][i][l].strip('-='), sent['ge'][i][l].strip('-='), ps, speaker))
                        except:
                            print(doc)
                            pprint(sent)
                    morphs.append(tuple(['END', 'END', 'END', speaker]))
    with open('by_speaker/{}_morphemes.pickle'.format(corpus), 'wb') as f:
        pickle.dump(morphs, f)
#     pprint(set(errors))
#     print(len(morphs))
#     pprint(morphs[:15])

`{'Amganov_EI_pear_story.txt', в \ft прописано, когда не первый, но с глоссами только первый
 'Amganovy_family.txt', – отмечается только EPA которая вторая в шапке
 'Amganovy_historical.txt', – то же
 'Amganovy_rybalka_tabun.txt', – АААААА иногда оба отмечаются, иногда нет, иногда целыми именем (сокр): ААААА
 'EPA_childhood_memory.txt', - на самом деле только EPA но указаны двое она вторая
 'Ichanga_Adukanov_museum.txt', – ААААААА реплики одного пишут в одну реплику с другим и АААА и тут просто фамилией
 'Tylkanova_JE_igodnyj_14-06-2009_NA.txt'}
{'Krivoshapkin_DM_Segen.txt',
 'Krivoshapkin_Misha_school.txt',
 'Nikitin_Mitja_pearstory.txt',
 'RDA_TPK_death.txt',
 'RDA_TPK_names_origins.txt',
 'Suzopova_Olja_eveny_tompo.txt',
 'Suzopova_Olja_pro_babushku.txt',
 'Suzopova_Olja_stado.txt',
 'Zaxarova_Zhenja_pear_story.txt',
 'beseda_180210_Sebjan.txt',
 'stado#10_Nikitin_Mitja_life_stado.txt'}`

In [12]:
for corp in corpora:
    extract_morphs(corp)

In [13]:
with open('by_speaker/Kamchatka_morphemes.pickle', 'rb') as f:
    kam_mor = pickle.load(f)
with open('by_speaker/Sebjan_morphemes.pickle', 'rb') as f:
    seb_mor = pickle.load(f)

In [14]:
from collections import Counter

seb_count = Counter(seb_mor)
kam_count = Counter(kam_mor)

kam_pd = pd.DataFrame({
                       'morpheme': [x[0] for x in kam_count if 'ROOT' not in x and 'END' not in x], 
                       'gloss': [x[1] for x in kam_count if 'ROOT' not in x and 'END' not in x],
                       'pos': [x[2] for x in kam_count if 'ROOT' not in x and 'END' not in x],
                       'speaker': [x[3] for x in kam_count if 'ROOT' not in x and 'END' not in x],
                       'count': [kam_count[x] for x in kam_count if 'ROOT' not in x and 'END' not in x]
    })

seb_pd = pd.DataFrame({
                       'morpheme': [x[0] for x in seb_count if 'ROOT' not in x and 'END' not in x], 
                       'gloss': [x[1] for x in seb_count if 'ROOT' not in x and 'END' not in x],
                       'pos': [x[2] for x in seb_count if 'ROOT' not in x and 'END' not in x],
                       'speaker': [x[3] for x in seb_count if 'ROOT' not in x and 'END' not in x],
                       'count': [seb_count[x] for x in seb_count if 'ROOT' not in x and 'END' not in x]
    })

df1 = pd.merge(seb_pd, kam_pd, how='outer', on=['morpheme', 'gloss', 'pos', 'speaker', 'count'])

In [30]:
sum(df1.loc[df1['morpheme']=='WEːČ'].loc[df1['pos']=='v'].loc[df1['gloss']=='gnr']['count'])

2232.0

In [33]:
morphemes = set(list(map(lambda x: x[:-1], seb_mor+kam_mor)))
speakers = set(list(map(lambda x: x[-1], seb_mor+kam_mor)))    
# morphemes = list(morphemes)
# df2 = pd.DataFrame({'morpheme': [], 'gloss': [], 'pos': []})

# for i in range(len(morphemes)):
#     sub = df1.loc[morphemes[i]]
#     df2.loc[i] = 

In [54]:
len(df1)

6184

In [71]:
# здесь _очень_долго_работающая_ функция которая считает по спикерам всё
for sp in speakers:
    df1[sp] = [0]*len(df1)

def make_speaker_table(df1):
    df2 = pd.DataFrame(columns=df1.columns)
    for i in range(len(df1)):
        if i%1000==0:
            print(i)
        if df1.loc[i]['count']>15:
            row = df1.loc[i]
            cs = row['speaker']
        #     print(df1.loc[i][cs])
            row[cs] = row['count']
            df2.loc[i] = row
    return df2

df2 = make_speaker_table(df1)

0


/Users/user/myproject/venv/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


1000
2000
3000
4000
5000
6000


In [72]:
df2[['count', 'speaker', 'RDA']].head()

,count,speaker,RDA
0,31.0,RDA,31.0
4,36.0,?,0.0
8,44.0,?,0.0
14,61.0,?,0.0
36,52.0,RDA,52.0


In [78]:
# здесь делаю так чтобы можно было ресетить индекс

df2['ind'] = df2['morpheme'] +'-'+ df2['gloss'] +'-'+ df2['pos']
spdf = df2[['morpheme', 'gloss', 'pos', 'ind']]
df2 = df2.groupby(['ind']).sum()
df2 = df2.reset_index()
df2.head()

,ind,count,NAS,AVZ,SPK,rh,LNZ,RMS,EAK,LNZ? IAS,...,PMB,TEB,T,children,AMG,TDA,ASA,ZAS,NMK,TPK
0,(E)ldE-soc-v,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,(E)rEp-adjr-adv,75.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,(E)rEp-adjr-n,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,(E)rEp-adjr-rel.n,23.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,(LI)lrE-imp.2pl-v,66.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [79]:
speaker_count = Counter([x[-1] for x in seb_mor+kam_mor if x[0]=='ROOT'])

In [92]:
speaker_count['?']

47149

Так. Спокойствие. По очереди

In [85]:
# Себьян
# 49800
wds = [x[-1] for x in seb_mor if x[0]=='ROOT'] # массив в котором на месте каждого слова стоит его спикер
print(len(wds))
seb_speakers = Counter(wds) # словарь спикер: кол-во слов
seb_parts = {x: seb_speakers[x]/len(wds) for x in seb_speakers} # слврь спикер: доля спикера (кол-во его слов на всего в корпусе)

seb_df = df2[list(seb_speakers.keys())]
seb_df['count'] = list(map(lambda i: sum(seb_df.loc[i]), range(len(seb_df))))
seb_df['morph'] = df2['ind']
# seb = df2[list(seb_speakers.keys())+['count']]
seb_df.head()

49800


/Users/user/myproject/venv/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/user/myproject/venv/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


,?,AVZ,MKK,VAK,SPK,TPA,NAS,TDA,LNZ,BP,...,AXK,rh,IAS,IAS,LAT,XAK,ZAS,TPK,count,morph
0,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,(E)ldE-soc-v
1,75.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,75.0,(E)rEp-adjr-adv
2,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,(E)rEp-adjr-n
3,23.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.0,(E)rEp-adjr-rel.n
4,66.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,66.0,(LI)lrE-imp.2pl-v


In [88]:
seb_df.loc[seb_df['count']!=seb_df['?']]

,?,AVZ,MKK,VAK,SPK,TPA,NAS,TDA,LNZ,BP,...,AXK,rh,IAS,IAS,LAT,XAK,ZAS,TPK,count,morph
6,106.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,127.0,(W)Ul-indef-pron
8,898.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1163.0,(dU)LE-loc-n
9,780.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33.0,0.0,...,0.0,0.0,21.0,0.0,0.0,0.0,0.0,0.0,1068.0,(dU)LE-loc-pron
16,22.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,38.0,(h)E-interj-v
19,237.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,278.0,B-med-v
20,1313.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27.0,0.0,...,0.0,0.0,17.0,0.0,0.0,0.0,0.0,0.0,1468.0,D-prog-v
24,330.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,351.0,DE-ptl-n
26,283.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,300.0,DE-ptl-pron
28,362.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,401.0,DE-purp.cvb-v
31,136.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,166.0,DI-adjr-n


In [140]:
dps = []

for i in range(len(seb_df)):
    line = seb_df.loc[i]
    exp = [seb_parts[x] for x in seb_df if x in seb_speakers]
    obv = [line[x]/line['count'] for x in seb_df if x in seb_speakers]
    if sum(obv)<0.999:
        print('Error in {}'.format(i))
    dp = sum(np.abs(np.array(obv)-np.array(exp)))/2
    dps.append(dp)

Окей, это были observed

In [142]:
seb_df['DP'] = dps
seb_df = seb_df.sort_values('DP', ascending=False)
seb_df.loc[seb_df['count']>10].head()

,VAK,AVK,rh,IAS,TPA,AEN,LNZ,EAK,SEN,children,...,NPZ,IVK,SPK,TDA,IDB,MK,NAS,count,morph,DP
330,1.0,7.0,0.0,1.0,1.0,4.0,8.0,43.0,7.0,0,...,27.0,71.0,0.0,0.0,4.0,0.0,18.0,868.0,L-pl-v,0.989558
337,0.0,0.0,0.0,0.0,0.0,4.0,1.0,1.0,0.0,0,...,2.0,0.0,2.0,0.0,0.0,0.0,3.0,36.0,LE-nr-n,0.987610
42,0.0,17.0,0.0,21.0,1.0,9.0,33.0,99.0,16.0,0,...,42.0,30.0,7.0,0.0,16.0,0.0,46.0,906.0,(dU)LE-loc-pron,0.987610
756,0.0,0.0,0.0,1.0,0.0,0.0,1.0,5.0,0.0,0,...,2.0,6.0,1.0,0.0,0.0,0.0,0.0,38.0,mdEs-sml-n,0.985582
599,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0,...,2.0,0.0,0.0,0.0,0.0,0.0,1.0,12.0,kE-emph-n,0.983715


In [143]:
seb_df.loc[seb_df['count']>10].to_excel('Sebjan_dp.xlsx')

In [146]:
# Камчатка
# 33112
wds = [x[-1] for x in kam_mor if x[0]=='ROOT'] # массив в котором на месте каждого слова стоит его спикер
print(len(wds))
kam_speakers = Counter(wds) # словарь спикер: кол-во слов
kam_parts = {x: kam_speakers[x]/len(wds) for x in kam_speakers} # слврь спикер: доля спикера (кол-во его слов на всего в корпусе)

kam_df = df2[list(kam_speakers.keys())]
kam_df['count'] = list(map(lambda i: sum(kam_df.loc[i]), range(len(kam_df))))
kam_df['morph'] = df2['ind']

dps = []
for i in range(len(kam_df)):
    line = kam_df.loc[i]
    exp = [kam_parts[x] for x in kam_df if x in kam_speakers]
    obv = [line[x]/line['count'] for x in kam_df if x in kam_speakers]
    if sum(obv)<0.999:
        print('Error in {}'.format(i))
    dp = sum(np.abs(np.array(obv)-np.array(exp)))/2
    dps.append(dp)

kam_df['DP'] = dps
kam_df = kam_df.sort_values('DP', ascending=False)
kam_df.loc[kam_df['count']>10].head()

kam_df.loc[kam_df['count']>10].to_excel('Kamchatka_dp.xlsx')

In [147]:
kam_df.loc[kam_df['count']>10].head()

,VIA,EPA,EIA,BP,JIP,NAT,JET,rh,TEB,GIK,...,RMS,ASA,AL,LGT,AGK,INB,RME,count,morph,DP
412,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,0.0,0.0,0.0,0.0,17.0,WEːČ-gnr.nonfut-v,0.898949
375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3.0,0,0.0,2.0,0.0,0.0,12.0,RUk-nr-n,0.788445
589,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0,0.0,1.0,0.0,1.0,14.0,k(E)-nr-n,0.753710
206,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0,7.0,0.0,0.0,2.0,17.0,El-emph-adv,0.738085
325,0.0,0.0,6.0,4.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0,0.0,0.0,0.0,0.0,25.0,L-pl-num,0.735958


In [151]:
docs = {'Amganov_EI_pear_story.txt', # в \ft прописано, когда не первый, но с глоссами только первый
 'Amganovy_family.txt', # – отмечается только EPA которая вторая в шапке
 'Amganovy_historical.txt', # – то же
 'Amganovy_rybalka_tabun.txt', # – АААААА иногда оба отмечаются, иногда нет, иногда целыми именем (сокр): ААААА
 'EPA_childhood_memory.txt', # - на самом деле только EPA но указаны двое она вторая
 'Ichanga_Adukanov_museum.txt', # – ААААААА реплики одного пишут в одну реплику с другим и АААА и тут просто фамилией
 'Tylkanova_JE_igodnyj_14-06-2009_NA.txt'}

hits = 0
for doc in docs:
    with open(os.path.join('Corpus_Text_Kamchatka_postagged', doc)) as f:
        text = f.read()
    hits += len(re.findall('WEːČ', text))
print(hits)

77


In [153]:
kam_df.sort_values('count', ascending=False).head()

,VIA,EPA,EIA,BP,JIP,NAT,JET,rh,TEB,GIK,...,RMS,ASA,AL,LGT,AGK,INB,RME,count,morph,DP
357,166.0,36.0,149.0,9.0,36.0,61.0,6.0,4.0,46.0,59.0,...,143.0,34.0,0,103.0,40.0,14.0,877.0,3130.0,R(E)-nonfut-v,0.107752
371,97.0,49.0,160.0,13.0,43.0,77.0,18.0,4.0,35.0,224.0,...,181.0,14.0,0,44.0,0.0,19.0,609.0,2829.0,RI-pst-v,0.140473
411,125.0,22.0,92.0,1.0,16.0,39.0,1.0,0.0,53.0,49.0,...,148.0,12.0,0,59.0,42.0,3.0,515.0,1910.0,WEːČ-gnr-v,0.152385
780,68.0,47.0,92.0,8.0,34.0,45.0,7.0,4.0,13.0,51.0,...,67.0,23.0,0,37.0,11.0,20.0,617.0,1879.0,n(I)-3sg-v,0.176244
106,57.0,23.0,106.0,2.0,16.0,45.0,7.0,1.0,33.0,85.0,...,90.0,7.0,0,29.0,4.0,20.0,430.0,1727.0,D-prog-v,0.110521


# Почему count не тот

In [19]:
with open('Kamchatka_morphemes.pickle', 'rb') as f:
    kam_mor_old = pickle.load(f)
with open('Sebjan_morphemes.pickle', 'rb') as f:
    seb_mor_old = pickle.load(f)
# with open('by_speaker/Kamchatka_morphemes.pickle', 'rb') as f:
#     kam_mor_new = pickle.load(f)
# with open('by_speaker/Sebjan_morphemes.pickle', 'rb') as f:
#     seb_mor_new = pickle.load(f)

def del_speakers(mors):
    mors = list(map(lambda x: x[:-1], mors))
    return mors

kam_mor_new, seb_mor_new = del_speakers(kam_mor), del_speakers(seb_mor)
print(kam_mor_old==kam_mor_new)
print(seb_mor_old==seb_mor_new)

False
False


In [22]:
Counter(kam_mor_old)==Counter(kam_mor_new)

True

Значит, всё нормально

In [32]:
Counter(kam_mor_old)[('WEːČ', 'gnr', 'v')]

1901

In [29]:
1901+332

2233